ToDo:
- [X] Pick more than one card
- [ ] Modify print statement based on Arcana
    - Major *doesn't need* "the" infront of it
        - Don't want "the The Tower"
        - how to deal with "Strength", does it print as "the Strength" 
    - Minor does 
        - "the Ten of Pentacles" 
- [ ] Add meaning? 
- [ ] Set random seed to 42
- [ ] Record - day, time, and item? Put them into a CSV? & Auto Export?
- [ ] Set up a Cron Job to randomly run?


In [1]:
import pandas as pd
import random
from math import comb
import datetime as dt
import os
from pathlib import Path

#current_time = dt.datetime.now()
#print(current_time)

In [2]:
csv_file_path = '/Users/jebbonanno/Documents/workspace/tarot_card_reader/tarot_cards.csv'

In [3]:
tarot_base_df = pd.read_csv(csv_file_path)

In [4]:
word_to_number = {
    "zero": 0, "one": 1, 'ace':1, "two": 2, "three": 3, "four": 4,
    "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9, 'ten':10
}

tarot_base_df['number_digit'] = tarot_base_df['Number'].apply(lambda x: word_to_number.get(x.lower(), None))
tarot_base_df['number_digit'].fillna(0, inplace=True)
tarot_base_df['number_digit'] = tarot_base_df['number_digit'].astype(int)

In [5]:
tarot_base_df.sample(10).sort_index()

,Name,Suit,Number,Order,number_digit
26,Five of Wands,Wands,Five,27,5
27,Seven of Wands,Wands,Seven,28,7
30,Nine of Wands,Wands,Nine,30,9
39,Four of Cups,Cups,Four,39,4
43,Eight of Cups,Cups,Eight,43,8
54,Five of Swords,Swords,Five,54,5
59,Ten of Swords,Swords,Ten,59,10
63,King of Swords,Swords,King,63,0
76,Page of Pentacles,Pentacles,Page,77,0
77,King of Pentacles,Pentacles,King,78,0


In [6]:
#probabilities 

prob_major_arcana = comb(22, 3) / comb(78, 3)
print(f"Probality all Major Arcana: {round((prob_major_arcana*100),2)}%")

prob_minor_arcana = 4*comb(14, 3) / comb(78, 3)
print(f"Probability all Minor Arcana: {round((prob_minor_arcana*100),2)}%")

prob_three_kind = (comb(4,3)*14)/comb(78,3)
print(f"Three of a Kind Probability: {round((prob_three_kind*100),2)}%")

prob_royals = comb(16, 3) / comb(78 , 3) 
print(f"Probability of all Royals (Page, Knight, Queen, King): {round((prob_royals*100),2)}%")

prob_flush = (4 * comb(14,3)) / comb(78 , 3) 
print(f"Probability of a Flush: {round((prob_flush*100),2)}%")

#prob_straight
single_straight = 4**3  # 4 choices per suit, 3 ranks in a straight
all_straight = 8 * single_straight #(8 possible straights)
prob_straight = (all_straight) / comb(78 , 3) 
print(f"Probability of a Straight: {round((prob_straight*100),2)}%")

Probality all Major Arcana: 2.02%
Probability all Minor Arcana: 1.91%
Three of a Kind Probability: 0.07%
Probability of all Royals (Page, Knight, Queen, King): 0.74%
Probability of a Flush: 1.91%
Probability of a Straight: 0.67%


### Condition Check

In [7]:
def condition_check(three_card_pull):
    
    court_list = ['Page', 'Knight', 'Queen', 'King']
    
    if three_card_pull['Number'].nunique() == 1:
        print("Three of a Kind")
        print(f"Three of a Kind Probability: {round((prob_three_kind*100),2)}%")
        
    elif three_card_pull['Suit'].nunique() == 1 and three_card_pull['Suit'][0] == 'Major Arcane':
        print("All Major Arcana")
        print(f"Probality all Major Arcana: {round((prob_major_arcana*100),2)}%")
    
    elif three_card_pull['Suit'].nunique() == 1:
        print("A Three Card Flush")
        print(f"Probability of all Flush: {round((prob_flush*100),2)}%")
        
    elif (three_card_pull['number_digit'][2] - three_card_pull['number_digit'][1] == 1 and 
          three_card_pull['number_digit'][1] - three_card_pull['number_digit'][0] == 1):
        print("A Straight")
        print(f"Probability of a Straight: {round((prob_straight*100),2)}%")
        
    elif three_card_pull['Number'].isin(court_list).all() == True:
        print("A Court Pull")
        print(f"Probability of all Royals (Page, Knight, Queen, King): {round((prob_royals*100),2)}%")
    # else:
    #     print('Nothing Special')

### Three Card Pull

In [8]:
def three_card_pull_function():
    three_card_pull_df = tarot_base_df.sample(3).reset_index(drop=True)

    # Extracting values
    past = three_card_pull_df.loc[0, "Name"]
    present = three_card_pull_df.loc[1, "Name"]
    future = three_card_pull_df.loc[2, "Name"]

    # Printing the values
    print(f"Past: {past}")
    print(f"Present: {present}")
    print(f"Future: {future}")

    # Call condition_check with the DataFrame
    condition_check(three_card_pull_df)

    # Return values as a dictionary
    return {"past": past, "present": present, "future": future}

In [9]:
def generate_tarot_pull():
    custom_col_order = ['date','time','past','present','future']
    
    new_tarot_pull = pd.DataFrame([three_card_pull_function()])
    new_tarot_pull['date'] = pd.Timestamp.now().date()
    new_tarot_pull['time'] = pd.Timestamp.now().strftime("%I:%M %p")
    new_tarot_pull = new_tarot_pull[custom_col_order]
    return new_tarot_pull

In [13]:
#File Exists 
#Daily Pull Explore

from pathlib import Path

custom_col_order = ['date','time','past','present','future']

file_name = 'tarot_record.csv'

file_path = Path.cwd() / file_name 

if file_path.exists():
    tarot_df = pd.read_csv(file_name, index_col = 0)
    
    new_pull = generate_tarot_pull()
    
    tarot_df = pd.concat([tarot_df, new_pull])
    tarot_df.to_csv(file_name, index=False)
    print("Added to Existing File")
    
else:
    starter_tarot_file = generate_tarot_pull()
    starter_tarot_file.to_csv(file_name, index=False)
    print("Created a New File")

Past: The Empress
Present: Nine of Swords
Future: Three of Wands
Added to Existing File


In [14]:
test = pd.read_csv(file_name)

In [15]:
test.head()

,time,past,present,future,date
0,09:37 AM,Nine of Pentacles,The Lovers,Page of Swords,NaN
1,09:37 AM,The Empress,Nine of Swords,Three of Wands,2025-02-19


In [ ]:
df.head()

In [ ]:
test = pd.concat([empty_df, df])

In [ ]:
test.head()

In [ ]:
#Old Way:
def three_card_pull_function():  # Renamed function to avoid conflict
    three_card_pull_df = tarot_df.sample(3).reset_index(drop=True)  # Renamed the variable

    for index, row in three_card_pull_df.iterrows():
        if index == 0:
            print(f"Past: {row['Name']}")
        elif index == 1:
            print(f"Present: {row['Name']}")
        elif index == 2:
            print(f"Future: {row['Name']}")
        else:
            print("Unexpected error")

    condition_check(three_card_pull_df)

In [ ]:
three_card_pull_function()

In [ ]:
#File Exists 
from pathlib import Path

file_name = 'tarot_record.csv'


file_path = Path.cwd() / file_name 

if file_path.exists():
    print("File exists")
else:
    column_names:



In [ ]:
pwd

In [ ]:
for i in range(0,1):
    three_card_pull_function()
    print()

### Single Pull

In [ ]:
def tarot_print_statement(card):
    if card[:3] == 'The':
        print(f"You drew {single_pick}.")
    else:
        print(f"You drew the {card}.")

In [ ]:
tarot_print_statement(tarot_df['Name'].sample(1).values[0])